In [37]:
import xarray as xr
import geopandas 
import config
import numpy as np
land = xr.open_dataset('~/Downloads/cLand_Emon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')
shape_file = geopandas.read_file(f'{config.SHAPEFILE_PATH}NABoreal.shp', crs="epsg:4326")
canada_shape_file = geopandas.read_file(f'{config.SHAPEFILE_PATH}lpr_000b21a_e/lpr_000b21a_e.shp')
soil = xr.open_dataset('~/Downloads/cSoil_Emon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')
litter = xr.open_dataset('~/Downloads/cLitter_Lmon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')
cwd = xr.open_dataset('~/Downloads/cCwd_Lmon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')
veg = xr.open_dataset('~/thesis/data/CESM/cVeg_Lmon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')


/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'cLand' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'cSoil' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'cLitter' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'cCwd' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: v

In [36]:
from shapely.geometry import mapping 
#need to check why mod makes this break ------ long180 = (long360 + 180) % 360 - 180
def netcdfToNumpy(netcdf_file,variable,shape_file,canada_shape_file):
    netcdf_file['lon'] = netcdf_file['lon'] - 360 if np.any(netcdf_file['lon'] > 180) else netcdf_file['lon']
    netcdf_file = netcdf_file.groupby('time.year').mean()
    netcdf_file = netcdf_file.sel(year=netcdf_file.year>=1949)
    netcdf_file = netcdf_file[variable]
    netcdf_file.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    netcdf_file.rio.write_crs("epsg:4326", inplace=True)
    clipped = netcdf_file.rio.clip(shape_file.geometry.apply(mapping), shape_file.crs,drop=True)
    clipped = clipped.rio.clip(canada_shape_file.geometry.apply(mapping), canada_shape_file.crs,drop=True)

    df = clipped.to_dataframe().reset_index()
    df = df[df[variable].notna()]
    return df[[variable]].values


In [38]:
soil_arr = netcdfToNumpy(soil,'cSoil',shape_file,canada_shape_file)
litter_arr = netcdfToNumpy(litter,'cLitter',shape_file,canada_shape_file)
cwd_arr = netcdfToNumpy(cwd,'cCwd',shape_file,canada_shape_file)
veg_arr = netcdfToNumpy(veg,'cVeg',shape_file,canada_shape_file)
totalc_arr = netcdfToNumpy(land,'cLand',shape_file,canada_shape_file)

In [39]:
soil_arr + litter_arr + cwd_arr + veg_arr

array([[ 30.139626],
       [ 22.81026 ],
       [ 22.800957],
       ...,
       [307.94034 ],
       [213.16951 ],
       [176.8477  ]], dtype=float32)

In [40]:
totalc_arr #aka cLand

array([[ 30.139626],
       [ 22.810259],
       [ 22.800955],
       ...,
       [307.94034 ],
       [213.16951 ],
       [176.8477  ]], dtype=float32)